In [1]:
import pandas as pd
import json
import numpy as np
from IPython.display import Markdown, HTML
from copy import deepcopy

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

## Load Stuff

In [4]:
raw_data = load_json('data_v2.json')

In [5]:
raw_data.keys()

dict_keys(['source', 'belts', 'pipes', 'buildings', 'miners', 'items', 'fluids', 'recipes', 'resources'])

In [6]:
key_name_fn = lambda x: x.lower().replace(' ', '-')

In [7]:
building_df = pd.DataFrame(raw_data['buildings'])

In [8]:
item_df = pd.DataFrame.from_dict(raw_data['items'])

In [9]:
fluid_df = pd.DataFrame.from_dict(raw_data['fluids'])

In [10]:
resource_df = pd.DataFrame.from_dict(raw_data['resources'])

In [11]:
recipe_df = pd.DataFrame.from_dict(raw_data['recipes'])

## Process recipe data

### Add power info

In [12]:
recipe_category_to_power = building_df.set_index('category')['power'].to_dict()
# average of fluctuating
recipe_category_to_power.update({'converting' : 250, 'encoding' : 1000})
recipe_df['power_rate'] = recipe_df['category'].map(recipe_category_to_power)

### Recipes for Base items - e.g. ore

In [13]:
base_items = ['Iron Ore', 'Limestone', 'Copper Ore', 'Bauxite', 'Caterium Ore', 'Coal','Raw Quartz', 'Sulfur', 'Uranium'] + ['Nitrogen Gas', 'Water', 'Crude Oil', 'SAM']
base_key_items = [key_name_fn(x) for x in base_items]
base_key_items_set = set(base_key_items)

In [14]:
list(recipe_df)

['name',
 'key_name',
 'category',
 'time',
 'ingredients',
 'products',
 'power_range',
 'power_rate']

In [15]:
base_recipes = pd.DataFrame([[it, key_it, 'base', 1/1200 if not key_it in {'water', 'crude-oil', 'nitrogen-gas'} else 1/600, [], [[key_it, 1]], np.nan, 0] for (it, key_it) in zip(base_items, base_key_items)], columns = list(recipe_df))
base_recipes.shape

(13, 8)

In [16]:
base_recipes['power_rate'] = [45, 45, 45, 45, 45, 45, 45, 45, 45, 150, 20, 40, 45]

In [17]:
recipe_df = pd.concat([recipe_df, base_recipes])

In [18]:
overclock = 'base'
sloop = 'base'

if not overclock in {'max', 'base'} and 'sloop' in {'max', 'base'}:
    raise ValueError

In [19]:
# 2.5**1.321928 from: https://satisfactory.wiki.gg/wiki/Clock_speed

if overclock == 'max':
    recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda x: [[y[0], y[1] * 2.5] for y in x])
    recipe_df['products'] = recipe_df['products'].apply(lambda x: [[y[0], y[1] * 2.5] for y in x])
    recipe_df['power_rate'] = recipe_df['power_rate'] * 2.5**1.321928
if sloop == 'max':
    recipe_df['products'] = recipe_df['products'].apply(lambda x: [[y[0], y[1] * 2] for y in x])
    recipe_df['power_rate'] = recipe_df['power_rate'] * 4
    

In [20]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.000000,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.000000,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.000000,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.000000,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30
4,Copper Ingot,copper-ingot,smelting1,2.000000,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4
...,...,...,...,...,...,...,...,...
8,Uranium,uranium,base,0.000833,[],"[[uranium, 1]]",NaN,45
9,Nitrogen Gas,nitrogen-gas,base,0.001667,[],"[[nitrogen-gas, 1]]",NaN,150
10,Water,water,base,0.001667,[],"[[water, 1]]",NaN,20
11,Crude Oil,crude-oil,base,0.001667,[],"[[crude-oil, 1]]",NaN,40


### load/process points data

In [85]:
points_df = pd.read_csv('points.csv')

In [86]:
points_df

,Points,Items
0,Cannot be sunk,"Alien Protein, Bacon Agaric, Beryl Nut, Bl..."
1,1,"Blue FICSMAS Ornament, FICSIT Coupon, FICSM..."
2,2,"FICSMAS Tree Branch, Iron Ingot, Limestone,..."
3,3,"Coal, Copper Ore, Leaves"
4,4,"FICSMAS Bow, Iron Rod"
...,...,...
120,"500,176",Assembly Director System
121,"538,976",Nuclear Pasta
122,"597,652",AI Expansion Server
123,"728,508",Thermal Propulsion Rocket


In [87]:
list(points_df)

['Points ', 'Items']

In [88]:
# split stuff as list by comma & explode
points_df['Items'] = points_df['Items'].apply(lambda x: x.split(','))
points_df = points_df.explode('Items').reset_index(drop=True)
points_df.rename({'Points ' : 'points', 'Items' : 'item'}, axis=1, inplace=True)
points_df = points_df[points_df['points'] != 'Cannot be sunk ']
points_df['item'] = points_df['item'].apply(lambda x: x.strip())
points_df['points'] = points_df['points'].apply(lambda x: int(x.replace(',', '')))

In [89]:
points_df

,points,item
22,1,Blue FICSMAS Ornament
23,1,FICSIT Coupon
24,1,FICSMAS Gift
25,1,Iron Ore
26,2,FICSMAS Tree Branch
...,...,...
174,500176,Assembly Director System
175,538976,Nuclear Pasta
176,597652,AI Expansion Server
177,728508,Thermal Propulsion Rocket


In [92]:
# points_df[points_df['item'] == 'SAM Fluctuator']

In [93]:



# ['alt-distilled-silica', 'alt-heavy-oil-residue', 'alt-polymer-resin', 'alumina-solution', 'ballistic-warp-drive', 'crude-oil', 'dark-matter-crystal', 'dark-matter-residue', 'diamonds', 'encased-plutonium-cell', 'excited-photonic-matter', 'ficsite-ingot', 'ficsite-trigon', 'ficsonium', 'ficsonium-fuel-rod', 'fuel', 'hatcher-remains', 'hog-remains', 'ionized-fuel', 'liquid-biofuel', 'neural-quantum-processor', 'nitric-acid', 'nitrogen-gas', 'non-fissile-uranium', 'packaged-ionized-fuel', 'packaged-rocket-fuel', 'plutonium-pellet', 'plutonium-waste', 'power-shard', 'reanimated-sam', 'rocket-fuel', 'sam', 'sam-fluctuator', 'singularity-cell', 'spitter-remains', 'stinger-remains', 'sulfuric-acid', 'superposition-oscillator', 'time-crystal', 'turbofuel', 'uranium-waste', 'water']


In [94]:
# missing_points = [
#     [597652, 'AI Expansion Server'],
#     [210, 'Alien Power Matrix'],
#     [2895334, 'Ballistic Warp Drive'],
#     [1780, 'Dark Matter Crystal'],
#     [240, 'Diamonds'],
#     [1936, 'Ficsite Ingot'],
#     [1291, 'Ficsite Trigon'],
#     [248034, 'Neural Quantum Processor'],
#     [5246, 'Packaged Ionized Fuel'],
#     [1028, 'Packaged Rocket Fuel'],
#     [160, 'Reanimated SAM'],
#     [20, 'SAM'],
#     [1968, 'SAM Fluctuator'],
#     [114675, 'Singularity Cell'],
#     [37292, 'Superposition Oscillator'],
#     [960, 'Time Crystal']
# ]
# points_df = pd.concat([points_df, pd.DataFrame.from_records(missing_points, columns = list(points_df))], axis=0).reset_index(drop=True)

In [95]:
garbage = ["Actual Snow",
"Candy Cane",
"FICSMAS Gift",
"Sweet Fireworks",
"Sparkly Fireworks",
"Copper FICSMAS Ornament",
"FICSMAS Ornament Bundle",
"Iron FICSMAS Ornament",
"Snowball",
"Red FICSMAS Ornament",
"FICSMAS Wonder Star",
"Fancy Fireworks",
"FICSMAS Bow",
'Portable Miner']

In [96]:
points_df = points_df[points_df['item'].apply(lambda x: not x in garbage)].reset_index(drop=True)

In [97]:
points_df['key_name'] = points_df['item'].apply(key_name_fn)

In [98]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.000000,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.000000,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.000000,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.000000,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30
4,Copper Ingot,copper-ingot,smelting1,2.000000,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4
...,...,...,...,...,...,...,...,...
8,Uranium,uranium,base,0.000833,[],"[[uranium, 1]]",NaN,45
9,Nitrogen Gas,nitrogen-gas,base,0.001667,[],"[[nitrogen-gas, 1]]",NaN,150
10,Water,water,base,0.001667,[],"[[water, 1]]",NaN,20
11,Crude Oil,crude-oil,base,0.001667,[],"[[crude-oil, 1]]",NaN,40


In [99]:
points_df

,points,item,key_name
0,1,Blue FICSMAS Ornament,blue-ficsmas-ornament
1,1,FICSIT Coupon,ficsit-coupon
2,1,Iron Ore,iron-ore
3,2,FICSMAS Tree Branch,ficsmas-tree-branch
4,2,Iron Ingot,iron-ingot
...,...,...,...
138,500176,Assembly Director System,assembly-director-system
139,538976,Nuclear Pasta,nuclear-pasta
140,597652,AI Expansion Server,ai-expansion-server
141,728508,Thermal Propulsion Rocket,thermal-propulsion-rocket


### Filter out unwanted things form poitns & recipes

In [111]:
ignore_things = [key_name_fn(x) for x in ['Hog Remains', 'Spitter Remains', 'Stinger Remains', 'Hatcher Remains', 'Leaves', 'Wood', 'Mycelia', 'Yellow Power Slug', 'Purple Power Slug', 'Blue Power Slug', 'Automated Miner', 'Alien DNA Capsule']]

In [112]:
print(recipe_df.shape)
recipe_df = recipe_df[recipe_df['products'].apply(lambda x: not any([y[0] in ignore_things for y in x]))].reset_index(drop=True)
recipe_df = recipe_df[recipe_df['ingredients'].apply(lambda x: not any([y[0] in ignore_things for y in x]))].reset_index(drop=True)
recipe_df = recipe_df[~recipe_df['key_name'].isin(ignore_things)].reset_index(drop=True)
recipe_df.shape

(272, 8)


(271, 8)

In [113]:
print(points_df.shape)
points_df = points_df[~points_df['key_name'].isin(ignore_things)].reset_index(drop=True)
points_df.shape

(140, 3)


(140, 3)

### Make points data & recipe data joinable

In [114]:
points_rename_mapper = {
    'encased-uranium-cell' : 'uranium-cell',
#     'biomass-(leaves)' : 'biomass-from-leaves',
#     'biomass-(mycelia)' : 'biomass-from-mycelia',
#     'biomass-(wood)' : 'biomass-from-wood',
    'gas-filter' : 'filter',
    'electromagnetic-control-rod' : 'em-control-rod',
    
}
points_df['key_name'] = points_df['key_name'].apply(lambda x: points_rename_mapper.get(x, x))

### Validate that everything is good
We need:  
* all ingredients/products that should be scoreable are scoreable
* WHAT ELSE DO WE NEED???

In [115]:
NO_SINK_ENTRY = set([key_name_fn(x) for x in ['Alien Protein', 'Encased Plutonium Cell', 'Ficsonium', 'Ficsonium Fuel Rod', 'Non Fissile Uranium', 'Plutonium Pellet', 'Plutonium Waste', 'Power Shard', 'Uranium Waste']] + fluid_df['key_name'].tolist())

In [116]:
all_ingredients = set(recipe_df['ingredients'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().unique())
all_products = set(recipe_df['products'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().unique())

In [117]:
all_scoreables = set(points_df['key_name'].tolist())

In [118]:
len(all_ingredients), len(all_products), len(all_scoreables)

(124, 138, 140)

In [119]:
fluid_df

,name,key_name,tier
0,Water,water,3
1,Crude Oil,crude-oil,5
2,Fuel,fuel,5
3,Turbofuel,turbofuel,6
4,Heavy Oil Residue,heavy-oil-residue,5
5,Liquid Biofuel,liquid-biofuel,5
6,Alumina Solution,alumina-solution,7
7,Sulfuric Acid,sulfuric-acid,7
8,Nitrogen Gas,nitrogen-gas,8
9,Nitric Acid,nitric-acid,8


In [120]:
missing_scores = (all_ingredients | all_products) - all_scoreables - NO_SINK_ENTRY

In [121]:
if len(missing_scores):
    print('MISSING: ', missing_scores)
    raise ValueError

### Setup/Solve LP!

In [122]:
"""
how on earth do I make a linear program again???

scipy API:
    params: A_ub, b_ub, A_eq, b_eq, (lb, ub)
    integrality: [0,3]; each int is method of considering problem; 3:= integers & 0 acceptable

    min c.T @ X
    s.t.
    A_{ub}x <= b_{ub}
    A_{eq}x == b_{eq}
    lb <= x <= ub


X: baseline: how many of each item am I making
C: associated score

Issue:
    items get used as ingredients don't get scored, but you'd want to use them potentially
    So need to duplicate all items:
        [items1, items2]
        items

"""


"\nhow on earth do I make a linear program again???\n\nscipy API:\n    params: A_ub, b_ub, A_eq, b_eq, (lb, ub)\n    integrality: [0,3]; each int is method of considering problem; 3:= integers & 0 acceptable\n\n    min c.T @ X\n    s.t.\n    A_{ub}x <= b_{ub}\n    A_{eq}x == b_{eq}\n    lb <= x <= ub\n\n\nX: baseline: how many of each item am I making\nC: associated score\n\nIssue:\n    items get used as ingredients don't get scored, but you'd want to use them potentially\n    So need to duplicate all items:\n        [items1, items2]\n        items\n\n"

In [123]:
base_inputs_test = {
    'iron-ore' : 1000,
}
base_inputs_irl = {
    'coal' : 42300,
    'crude-oil' : 12600,
    'nitrogen-gas' : 12000,
    'bauxite' : 12300,
    'copper-ore': 36900,
    'caterium-ore' : 15000,
    'iron-ore' : 92100,
    'uranium' : 2100,
    'raw-quartz' : 13500,
    'sam' : 10200,
    'limestone' : 69300,
    'sulfur' : 10800,
    'water' : 131250,
    'excited-photonic-matter' : 10000,
}

for key in base_items:
    key = key.lower().replace(' ', '-')
    if key in base_inputs_irl:
        continue
    base_inputs_irl[key] = 0


base_inputs_irl = base_inputs_irl | {x_processed : 0 for x in base_items if (not (x_processed:=x.lower().replace(' ', '-'))) in base_inputs_irl}

In [124]:
from scipy.optimize import linprog

In [125]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.000000,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.000000,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.000000,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.000000,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30
4,Copper Ingot,copper-ingot,smelting1,2.000000,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4
...,...,...,...,...,...,...,...,...
266,Uranium,uranium,base,0.000833,[],"[[uranium, 1]]",NaN,45
267,Nitrogen Gas,nitrogen-gas,base,0.001667,[],"[[nitrogen-gas, 1]]",NaN,150
268,Water,water,base,0.001667,[],"[[water, 1]]",NaN,20
269,Crude Oil,crude-oil,base,0.001667,[],"[[crude-oil, 1]]",NaN,40


In [126]:
points_df

,points,item,key_name
0,1,Blue FICSMAS Ornament,blue-ficsmas-ornament
1,1,FICSIT Coupon,ficsit-coupon
2,1,Iron Ore,iron-ore
3,2,FICSMAS Tree Branch,ficsmas-tree-branch
4,2,Iron Ingot,iron-ingot
...,...,...,...
135,500176,Assembly Director System,assembly-director-system
136,538976,Nuclear Pasta,nuclear-pasta
137,597652,AI Expansion Server,ai-expansion-server
138,728508,Thermal Propulsion Rocket,thermal-propulsion-rocket


In [127]:
import math

In [128]:
#gpt4o couldn't write this function, but claude could
def style_column(col):
    """
    Style a column with color gradient from red (min) to green (max)
    Args:
        col: pandas Series with numeric values
    Returns:
        List of CSS background-color styles
    """
    # Normalize values between 0 and 1
    if pd.isnull(col).all():
        return [''] * col.shape[0]
    min_val = col.min()
    max_val = col.max()
    norm = (col - min_val) / (max_val - min_val)
    
    # Convert to RGB colors (red to green)
    def get_color(value):
        r = int(255 * (1 - value))
        g = int(255 * value)
        return f'background-color: rgba({r}, {g}, 0, 0.5)'
    
    return [get_color(x) for x in norm]

In [129]:
def lp_solve(pdf, rdf, base_inputs, sort_by_col):
    valid_sort_by_col= ['qty_produce', 'score_output', 'power_work']
    if not sort_by_col in valid_sort_by_col + ['all']:
        raise ValueError(f'sort by col must be in {valid_sort_by_col} or "all"')
    
    pdf = pdf.copy()
    point_mapper = pdf.set_index('key_name')['points'].to_dict()
    
    rdf = rdf.copy()
    if not rdf['key_name'].nunique() == rdf.shape[0]:
        raise ValueError('"key_name" in rrecipe df assumed to be unique, but its not!!!')
        
    all_recipe_items = sorted(list(set(rdf['ingredients'].explode().dropna().apply(lambda x: x[0]).tolist() + rdf['products'].explode().dropna().apply(lambda x: x[0]).tolist())))
    all_recipe_items_set = set(all_recipe_items)
        
    base_inputs = {k : base_inputs[k] for k in sorted(base_inputs) if k in all_recipe_items_set}
#     print(base_inputs)
#     sys.exit()
    
    #pdf = points_df, rdf=recipe_df
    
    
    
    
    
    num_items_with_points = np.sum([item in point_mapper for item in all_recipe_items])
    trash_df = pd.DataFrame.from_records([[f'{item} Trash', f'{item}-trash', 'sink', 0, [[item, 1]], [], np.nan, 0] for item in all_recipe_items if item in point_mapper], columns = list(rdf))
    rdf = pd.concat([rdf, trash_df], axis=0).reset_index(drop=True)    
    rdf['id'] = np.arange(rdf.shape[0])
    
    recipe_to_col_idx = {name : i for i, name in enumerate(rdf['key_name'].tolist())}

#     display(Markdown('## Inputs'))
#     display(pdf)
#     display(rdf)
#     sys.exit()
    
    
    recipe_constraint_row_order = {ing : i for i, ing in enumerate(all_recipe_items)}
    inv_recipe_constraint_row_order = {ing : i for i, ing in enumerate(all_recipe_items)}
    
    constraint_matrix = np.zeros((len(all_recipe_items), rdf.shape[0]))
    constraint_values = np.zeros(len(all_recipe_items))
    
    
    def build_recipe_constraints(row):
        ingr_list = row['ingredients']
        products = row['products']
        col_idx = row['id']
        
        if len(ingr_list):
            for ingr, qty in ingr_list:
                constraint_matrix[recipe_constraint_row_order[ingr], col_idx] = qty
        if len(products):
            for prod, qty in products:
                constraint_matrix[recipe_constraint_row_order[prod], col_idx] = -qty
        
        """
        if not len(ingr_list):
            base_amount = base_inputs[row['key_name']]
            constraint_values[recipe_constraint_row_order[row['key_name']]] = base_amount
            constraint_matrix[recipe_constraint_row_order[row['key_name']], col_idx] = -1
        else:
            for ingr, qty in ingr_list:
                constraint_matrix[recipe_constraint_row_order[ingr], col_idx] = qty
            if len(products):
                for prod, qty in products:
                    constraint_matrix[recipe_constraint_row_order[prod], col_idx] = -qty
        """
                
    
    rdf.apply(build_recipe_constraints, axis=1)
    
#     base_ingredients = recipe_df[(recipe_df['ingredients'].apply(len) == 0) & (recipe_df['power_rate'] == 0)]['key_name'].tolist()
    
    
    raw_material_constraints = np.zeros((len(base_inputs), rdf.shape[0]))
    raw_material_constraint_values = np.zeros(len(base_inputs))
    
    for i, (item, amount) in enumerate(base_inputs.items()):
        raw_material_constraints[i, recipe_to_col_idx[item]] = 1
        raw_material_constraint_values[i] = amount
    
    
#     print(raw_material_constraints)
#     print(raw_material_constraint_values)
#     sys.exit()


#     print(constraint_matrix.shape, constraint_values.shape)
    constraint_matrix = np.concatenate((constraint_matrix, raw_material_constraints), axis=0)
    constraint_values = np.concatenate((constraint_values, raw_material_constraint_values), axis=0)
#     print(constraint_matrix.shape, constraint_values.shape)
#     sys.exit()
    
    
            
            
#     print(constraint_matrix)
#     print(constraint_values)

    maximize_vec = np.array([0] *(rdf.shape[0] - num_items_with_points) + [point_mapper[item] for item in all_recipe_items if item in point_mapper])

    constraint_df = pd.DataFrame(np.concatenate((np.concatenate((constraint_matrix, constraint_values[:,np.newaxis]), axis=1), np.concatenate((maximize_vec, [np.nan]))[np.newaxis,:]), axis=0), index = all_recipe_items + list(base_inputs.keys()) + ['**POINTS**'], columns = rdf['key_name'].tolist() + ['CONSTRAINT_VALUE'])#.reset_index()
#     constraint_df.rename({'index' : 'resource'}, axis=1)
    
    display(Markdown('## Constraints'))
#     styled = constraint_df.style.apply(highlight_last_row_and_col, axis=None)
    display(constraint_df)
#     sys.exit()
#     display(constraint_df.style.apply(style_column))
    
    

    lp_res = linprog(-maximize_vec, A_ub=constraint_matrix, b_ub=constraint_values, A_eq=None, b_eq=None, bounds=(0, None), method='highs', callback=None, options={'disp' : True}, x0=None, integrality=3)
    
    values = lp_res.x
    
    rdf['qty_produce'] = values
    
    # "[:-6]" --> removes "-trash" suffix
    rdf['score_output'] = rdf.apply(lambda x: 0 if not x['key_name'].endswith('-trash') else x['qty_produce'] * point_mapper[x['key_name'][:-6]], axis=1)
    rdf['power_work'] = rdf['qty_produce'] * rdf['power_rate']
    
#     display(rdf)
    
    display(Markdown('## Output'))
    if sort_by_col == 'all':
        for c in valid_sort_by_col:
            display(Markdown(f'#### {c}'))
            display(rdf.sort_values(c, ascending=False).style.apply(style_column, subset=[c]))
    else:
        display(rdf.sort_values(sort_by_col, ascending=False).style.apply(style_column, subset=[sort_by_col]))
    
    return lp_res, rdf
    

In [133]:
eg_points = points_df[points_df['key_name'].isin({'iron-ore', 'iron-ingot', 'iron-plate', 'iron-rod'})].reset_index(drop=True).copy()
eg_recipes = recipe_df[recipe_df['key_name'].isin({'iron-ingot', 'iron-plate', 'iron-ore', 'iron-rod'})].reset_index(drop=True).copy()

eg_points = points_df[points_df['key_name'].isin({'iron-ore', 'iron-ingot', 'iron-plate'})].reset_index(drop=True).copy()
eg_recipes = recipe_df[recipe_df['key_name'].isin({'iron-ingot', 'iron-plate', 'iron-ore'})].reset_index(drop=True).copy()

# eg_points = points_df[points_df['key_name'].isin({'iron-ore', 'iron-ingot'})].reset_index(drop=True).copy()
# eg_recipes = recipe_df[recipe_df['key_name'].isin({'iron-ingot', 'iron-ore'})].reset_index(drop=True).copy()

In [134]:
_ = lp_solve(eg_points, eg_recipes, base_inputs_test, sort_by_col = 'all')

## Constraints

,iron-ingot,iron-plate,iron-ore,iron-ingot-trash,iron-ore-trash,iron-plate-trash,CONSTRAINT_VALUE
iron-ingot,-1.0,3.0,0.0,1.0,0.0,0.0,0.0
iron-ore,1.0,0.0,-1.0,0.0,1.0,0.0,0.0
iron-plate,0.0,-2.0,0.0,0.0,0.0,1.0,0.0
iron-ore,0.0,0.0,1.0,0.0,0.0,0.0,1000.0
**POINTS**,0.0,0.0,0.0,2.0,1.0,6.0,NaN


## Output

#### qty_produce

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
0,Iron Ingot,iron-ingot,smelting1,2.000000,"[['iron-ore', 1]]","[['iron-ingot', 1]]",nan,4,0,1000.000000,0.000000,4000.000000
2,Iron Ore,iron-ore,base,0.000833,[],"[['iron-ore', 1]]",nan,45,2,1000.000000,0.000000,45000.000000
5,iron-plate Trash,iron-plate-trash,sink,0.000000,"[['iron-plate', 1]]",[],nan,0,5,666.000000,3996.000000,0.000000
1,Iron Plate,iron-plate,crafting1,6.000000,"[['iron-ingot', 3]]","[['iron-plate', 2]]",nan,4,1,333.000000,0.000000,1332.000000
3,iron-ingot Trash,iron-ingot-trash,sink,0.000000,"[['iron-ingot', 1]]",[],nan,0,3,1.000000,2.000000,0.000000
4,iron-ore Trash,iron-ore-trash,sink,0.000000,"[['iron-ore', 1]]",[],nan,0,4,0.000000,0.000000,0.000000


#### score_output

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
5,iron-plate Trash,iron-plate-trash,sink,0.000000,"[['iron-plate', 1]]",[],nan,0,5,666.000000,3996.000000,0.000000
3,iron-ingot Trash,iron-ingot-trash,sink,0.000000,"[['iron-ingot', 1]]",[],nan,0,3,1.000000,2.000000,0.000000
1,Iron Plate,iron-plate,crafting1,6.000000,"[['iron-ingot', 3]]","[['iron-plate', 2]]",nan,4,1,333.000000,0.000000,1332.000000
0,Iron Ingot,iron-ingot,smelting1,2.000000,"[['iron-ore', 1]]","[['iron-ingot', 1]]",nan,4,0,1000.000000,0.000000,4000.000000
2,Iron Ore,iron-ore,base,0.000833,[],"[['iron-ore', 1]]",nan,45,2,1000.000000,0.000000,45000.000000
4,iron-ore Trash,iron-ore-trash,sink,0.000000,"[['iron-ore', 1]]",[],nan,0,4,0.000000,0.000000,0.000000


#### power_work

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
2,Iron Ore,iron-ore,base,0.000833,[],"[['iron-ore', 1]]",nan,45,2,1000.000000,0.000000,45000.000000
0,Iron Ingot,iron-ingot,smelting1,2.000000,"[['iron-ore', 1]]","[['iron-ingot', 1]]",nan,4,0,1000.000000,0.000000,4000.000000
1,Iron Plate,iron-plate,crafting1,6.000000,"[['iron-ingot', 3]]","[['iron-plate', 2]]",nan,4,1,333.000000,0.000000,1332.000000
3,iron-ingot Trash,iron-ingot-trash,sink,0.000000,"[['iron-ingot', 1]]",[],nan,0,3,1.000000,2.000000,0.000000
4,iron-ore Trash,iron-ore-trash,sink,0.000000,"[['iron-ore', 1]]",[],nan,0,4,0.000000,0.000000,0.000000
5,iron-plate Trash,iron-plate-trash,sink,0.000000,"[['iron-plate', 1]]",[],nan,0,5,666.000000,3996.000000,0.000000


In [ ]:
# debugging_input= pd.concat([recipe_df.head(4), recipe_df[recipe_df['key_name'].isin({'copper-ore', 'iron-ore', 'limestone', 'water'})]])

In [ ]:
# debugging_input

In [135]:
res = lp_solve(points_df, recipe_df, base_inputs_irl, sort_by_col = 'all')

## Constraints

,iron-ingot,alt-iron-ingot,alt-pure-iron-ingot,alt-leached-iron-ingot,copper-ingot,alt-copper-ingot,alt-pure-copper-ingot,alt-leached-copper-ingot,steel-ingot,alt-steel-ingot,...,thermal-propulsion-rocket-trash,time-crystal-trash,turbo-motor-trash,turbo-rifle-ammo-trash,uranium-trash,uranium-cell-trash,uranium-fuel-rod-trash,versatile-framework-trash,wire-trash,CONSTRAINT_VALUE
adaptive-control-unit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ai-expansion-server,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ai-limiter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alclad-aluminum-sheet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alien-power-matrix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10200.0
sulfur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10800.0
uranium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2100.0
water,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131250.0


## Output

#### qty_produce

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
268,Water,water,base,0.001667,[],"[['water', 1]]",nan,20,268,131250.000000,0.000000,2625000.000000
258,Iron Ore,iron-ore,base,0.000833,[],"[['iron-ore', 1]]",nan,45,258,92100.000000,0.000000,4144500.000000
259,Limestone,limestone,base,0.000833,[],"[['limestone', 1]]",nan,45,259,69300.000000,0.000000,3118500.000000
263,Coal,coal,base,0.000833,[],"[['coal', 1]]",nan,45,263,42300.000000,0.000000,1903500.000000
22,Cable,cable,crafting1,2.000000,"[['wire', 2]]","[['cable', 1]]",nan,4,22,37600.000000,0.000000,150400.000000
260,Copper Ore,copper-ore,base,0.000833,[],"[['copper-ore', 1]]",nan,45,260,36900.000000,0.000000,1660500.000000
121,Alternate: Pure Aluminum Ingot,alt-aluminum-ingot,smelting1,2.000000,"[['aluminum-scrap', 2]]","[['aluminum-ingot', 1]]",nan,4,121,20280.000000,0.000000,81120.000000
262,Caterium Ore,caterium-ore,base,0.000833,[],"[['caterium-ore', 1]]",nan,45,262,15000.000000,0.000000,675000.000000
264,Raw Quartz,raw-quartz,base,0.000833,[],"[['raw-quartz', 1]]",nan,45,264,13500.000000,0.000000,607500.000000
269,Crude Oil,crude-oil,base,0.001667,[],"[['crude-oil', 1]]",nan,40,269,12600.000000,0.000000,504000.000000


#### score_output

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
281,ballistic-warp-drive Trash,ballistic-warp-drive-trash,sink,0.000000,"[['ballistic-warp-drive', 1]]",[],nan,0,281,58.000000,167929372.000000,0.000000
279,assembly-director-system Trash,assembly-director-system-trash,sink,0.000000,"[['assembly-director-system', 1]]",[],nan,0,279,188.000000,94033088.000000,0.000000
272,ai-expansion-server Trash,ai-expansion-server-trash,sink,0.000000,"[['ai-expansion-server', 1]]",[],nan,0,272,102.000000,60960504.000000,0.000000
332,nuke-nobelisk Trash,nuke-nobelisk-trash,sink,0.000000,"[['nuke-nobelisk', 1]]",[],nan,0,332,215.000000,4214000.000000,0.000000
382,uranium-cell Trash,uranium-cell-trash,sink,0.000000,"[['uranium-cell', 1]]",[],nan,0,382,190.000000,27930.000000,0.000000
293,computer Trash,computer-trash,sink,0.000000,"[['computer', 1]]",[],nan,0,293,1.000000,8352.000000,0.000000
344,packaged-water Trash,packaged-water-trash,sink,0.000000,"[['packaged-water', 1]]",[],nan,0,344,16.000000,2080.000000,0.000000
384,versatile-framework Trash,versatile-framework-trash,sink,0.000000,"[['versatile-framework', 1]]",[],nan,0,384,1.000000,1176.000000,0.000000
345,petroleum-coke Trash,petroleum-coke-trash,sink,0.000000,"[['petroleum-coke', 1]]",[],nan,0,345,3.000000,60.000000,0.000000
292,compacted-coal Trash,compacted-coal-trash,sink,0.000000,"[['compacted-coal', 1]]",[],nan,0,292,1.000000,28.000000,0.000000


#### power_work

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
258,Iron Ore,iron-ore,base,0.000833,[],"[['iron-ore', 1]]",nan,45,258,92100.000000,0.000000,4144500.000000
259,Limestone,limestone,base,0.000833,[],"[['limestone', 1]]",nan,45,259,69300.000000,0.000000,3118500.000000
268,Water,water,base,0.001667,[],"[['water', 1]]",nan,20,268,131250.000000,0.000000,2625000.000000
223,Excited Photonic Matter,excited-photonic-matter,converting,3.000000,[],"[['excited-photonic-matter', 10]]",nan,250,223,10000.000000,0.000000,2500000.000000
263,Coal,coal,base,0.000833,[],"[['coal', 1]]",nan,45,263,42300.000000,0.000000,1903500.000000
267,Nitrogen Gas,nitrogen-gas,base,0.001667,[],"[['nitrogen-gas', 1]]",nan,150,267,12000.000000,0.000000,1800000.000000
260,Copper Ore,copper-ore,base,0.000833,[],"[['copper-ore', 1]]",nan,45,260,36900.000000,0.000000,1660500.000000
262,Caterium Ore,caterium-ore,base,0.000833,[],"[['caterium-ore', 1]]",nan,45,262,15000.000000,0.000000,675000.000000
218,Time Crystal,time-crystal,converting,10.000000,"[['diamonds', 2]]","[['time-crystal', 1]]",nan,250,218,2612.000000,0.000000,653000.000000
264,Raw Quartz,raw-quartz,base,0.000833,[],"[['raw-quartz', 1]]",nan,45,264,13500.000000,0.000000,607500.000000


In [ ]:
res[1]

In [ ]:
61249614.24 / 122.4561239 

In [ ]:
109813664.000000 / 202.000000

In [ ]:
335_687_462 / 480000000

In [ ]:
61_249_614.24  + 347440080 

In [ ]:
 109813664.000000